In [1]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import geopandas
import re
from collections import defaultdict

In [2]:
df = geopandas.read_file("water_mains_with_roads.zip")

In [3]:
df.columns

Index(['OBJECTID_r', 'segment_id', 'segment_na', 'from_segme', 'to_segment',
       'pvmt_ratin', 'curb_ratin', 'index_righ', 'OBJECTID_b', 'MainID',
       'street_nam', 'street_typ', 'StreetSegm', 'break_date', 'break_year',
       'break_mont', 'break_day', 'pipe_size', 'pipe_depth', 'soil_type',
       'materials_', 'TotalHours', 'street_dir', 'Location', 'time_recei',
       'time_at_sc', 'time_off', 'time_on', 'num_bldgs_', 'frost_dept',
       'FacilityID', 'MainRetire', 'BreakID', 'BreakType', 'OBJECTID_a',
       'InstallYea', 'Lifecycle', 'Diameter', 'Material', 'Bagged',
       'LiningType', 'Shape_STLe', 'geometry'],
      dtype='object')

In [4]:
df = df.drop_duplicates(subset = "OBJECTID_b",keep = "first").copy()
df["break_mont"] = df.break_date.str.extract("(\d{4})-(\d{2})-(\d{2})")[1].astype(float)
df["break_day"] = df.break_date.str.extract("(\d{4})-(\d{2})-(\d{2})")[2].astype(float)

In [5]:
df = df[df["break_year"]>1000].copy()

In [6]:
#return True if broken prior or at that year given
def has_broken_prior(small_df, year):
    if len(small_df)==0:
        return False
    if small_df["break_year"].min() <= year:
        return True
    return False

In [7]:
#counting from the beginning of the year, check if it breaks within that interval
#2001, 3 return True if break happens in either 2001,2002,2003
def breaks_in_interval(all_yrs, year,interval):
    for i in range(year, year+interval):
        if i in all_yrs:
            return True
    return False

In [8]:
a = ['OBJECTID_r','segment_id','segment_na','from_segme','to_segment','pvmt_ratin','curb_ratin',
     'geometry','Install_year','CI','DI','SPUN','SAND']
a.append("prior_breaks")
a.append("y")

In [26]:
segment_list = list(set(df["OBJECTID_r"]))
ml_df = pd.DataFrame(columns = a)
yes = 0
no = 0

In [27]:
def add_to_ml_df(sgm,all_records,year1,year2):
    global already_broken
    global ml_df
    row = all_records[['OBJECTID_r','segment_id','segment_na','from_segme','to_segment','pvmt_ratin','curb_ratin','geometry']].iloc[0,]
    row["Install_year"] = all_records["InstallYea"].unique().mean()
    for m in ["CI","DI","SPUN","SAND"]:
        row[m] = int(m in all_records["Material"].values)
    last_break_year = None
    for year in range(year1,year2):
        y = breaks_in_interval(all_records["break_year"].values,year,interval)
        temp = all_records[all_records["break_year"]==year]
        if len(temp)!=0:
            last_break_year = year
            for idx in temp.index:
                ml_df = ml_df.append(pd.concat([row,pd.Series({"year":year,"last_break_yr":last_break_year,"prior_breaks":already_broken[sgm],"y":y})]),ignore_index = True)
                already_broken[sgm] += 1
        else:
            ml_df = ml_df.append(pd.concat([row,pd.Series({"year":year,"last_break_yr":last_break_year,"prior_breaks":already_broken[sgm],"y":y})]),ignore_index = True)

In [28]:
start_year = 1980
end_year = 2020
interval = 20
already_broken = defaultdict(int)
for sgm in segment_list:
    all_records = df[(df["OBJECTID_r"]==sgm) & (df["break_year"].notnull())].copy()
    all_records.sort_values(by = "break_year",inplace = True)
    for year in range(start_year,end_year+1-interval):
        if not has_broken_prior(all_records,year):
            if breaks_in_interval(all_records["break_year"].values,year+1,interval):
                yes +=1
                break
            else:
                no +=1
        else:
            add_to_ml_df(sgm,all_records,year,end_year+1-interval)
            break 

In [29]:
ml_df

,OBJECTID_r,segment_id,segment_na,from_segme,to_segment,pvmt_ratin,curb_ratin,geometry,Install_year,CI,DI,SPUN,SAND,prior_breaks,y,last_break_yr,year
0,746,08072,W SKYLINE DR,S HIGHLANDS AVE,E SKYLINE DR,4.0,0.0,LINESTRING (792803.8145548572 485679.681359891...,1958.000000,1,0,0,0,0,True,1980.0,1980.0
1,746,08072,W SKYLINE DR,S HIGHLANDS AVE,E SKYLINE DR,4.0,0.0,LINESTRING (792803.8145548572 485679.681359891...,1958.000000,1,0,0,0,1,True,1980.0,1981.0
2,746,08072,W SKYLINE DR,S HIGHLANDS AVE,E SKYLINE DR,4.0,0.0,LINESTRING (792803.8145548572 485679.681359891...,1958.000000,1,0,0,0,1,True,1980.0,1982.0
3,746,08072,W SKYLINE DR,S HIGHLANDS AVE,E SKYLINE DR,4.0,0.0,LINESTRING (792803.8145548572 485679.681359891...,1958.000000,1,0,0,0,1,True,1983.0,1983.0
4,746,08072,W SKYLINE DR,S HIGHLANDS AVE,E SKYLINE DR,4.0,0.0,LINESTRING (792803.8145548572 485679.681359891...,1958.000000,1,0,0,0,2,True,1983.0,1984.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2660,7792,07106,RIDGEWAY AVE,MENDOTA ST,711 FT NE OF MENDOTA ST,5.0,0.0,LINESTRING (837661.8373721254 499219.402311868...,1975.333333,1,1,1,0,3,True,1993.0,1996.0
2661,7792,07106,RIDGEWAY AVE,MENDOTA ST,711 FT NE OF MENDOTA ST,5.0,0.0,LINESTRING (837661.8373721254 499219.402311868...,1975.333333,1,1,1,0,3,True,1997.0,1997.0
2662,7792,07106,RIDGEWAY AVE,MENDOTA ST,711 FT NE OF MENDOTA ST,5.0,0.0,LINESTRING (837661.8373721254 499219.402311868...,1975.333333,1,1,1,0,4,True,1998.0,1998.0
2663,7792,07106,RIDGEWAY AVE,MENDOTA ST,711 FT NE OF MENDOTA ST,5.0,0.0,LINESTRING (837661.8373721254 499219.402311868...,1975.333333,1,1,1,0,5,True,1998.0,1999.0


In [30]:
yes

2154

In [31]:
no

9830

In [32]:
yes/no

0.21912512716174976

In [33]:
ml_df.to_csv("ML_20yr_dataset.csv")